<a href="https://colab.research.google.com/github/StarWanderer1337/GridMaster/blob/main/grid_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO

1. document

In [14]:
from gspread.worksheet import ValueRange

from google.auth import default
from google.colab import drive
from google.colab import auth

from typing import Union
from typing import List
from typing import Any

import pandas as pd

import gspread
import random

In [33]:
def read_sheet(all_values:Union[ValueRange, List[List[Any]]]) -> pd.DataFrame:
  try:
      # Extract the header row
      header = all_values[0]
      # Extract the data rows
      data_rows = all_values[1:]
      # Create a Pandas DataFrame
      df = pd.DataFrame(data_rows, columns=header)
      # Add a "id" column
      df.insert(0, "id", range(2, len(all_values) + 1))

      return df

  except Exception as err:
      print(f"Unexpected {err=}, {type(err)=}")
      return pd.DataFrame(data=None)

def update_sheet(sheet:gspread.worksheet.Worksheet, df:pd.DataFrame, ) -> bool:
  try:
    # Remove the previously created column id
    del df["id"]
    # Get headers and data into the same dataframe
    data = [df.columns.tolist()] + df.values.tolist()
    # Write data to cells
    sheet.update(range_name="A1", values=data)

    return True

  except Exception as err:
      print(f"Unexpected {err=}, {type(err)=}")
      return False

In [35]:
# Get credentials and create a client
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Connect to google drive
drive.mount("/content/drive", force_remount=True)

# Open a Google Sheet by title (replace with your sheet"s title)
sheet = gc.open("ping").worksheet(title="pung")

# Append row
sheet.append_row([f"p{random.randint(a=0, b=9)}ng"])

# Read data from cells & display
# - Use pandas functions to CRUD
df = read_sheet(all_values=sheet.get_all_values())

# Update row id 4
column_to_update = "pyng"
df.loc[(df["id"] == 4), column_to_update] = f"p{random.randint(a=0, b=9)}ng"

update_sheet(sheet=sheet, df=df)
display(read_sheet(all_values=sheet.get_all_values()))

Mounted at /content/drive


,id,pyng
0,2,pong
1,3,peng
2,4,p3ng
3,5,p8ng
4,6,p1ng
5,7,p0ng
6,8,p0ng


In [43]:
!git status

fatal: not a git repository (or any of the parent directories): .git
